In [3]:
import cv2

img = cv2.imread("final_track.mp4")
cap = cv2.VideoCapture("final_track.mp4")
ret, img = cap.read()

points = []

def click(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))
        print(x, y)

cv2.imshow("Click 4 pitch corners", img)
cv2.setMouseCallback("Click 4 pitch corners", click)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Camera points:", points)


572 455
1252 466
1273 647
574 644
Camera points: [(572, 455), (1252, 466), (1273, 647), (574, 644)]


In [5]:
import cv2

img = cv2.imread("Ground Image.png")
points = []

def click(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        points.append((x, y))
        print(x, y)

cv2.imshow("Click 4 pitch corners", img)
cv2.setMouseCallback("Click 4 pitch corners", click)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Top view points:", points)


299 288
803 304
797 650
254 659
Top view points: [(299, 288), (803, 304), (797, 650), (254, 659)]


In [7]:
import cv2
import numpy as np
import pandas as pd



In [9]:
# Camera frame points 
cam_pts = np.array([[572, 455], [1252, 466], [1273, 647], [574, 644]], dtype=np.float32)

# Top-view points
top_pts = np.array([[299, 288], [803, 304], [797, 650], [254, 659]], dtype=np.float32)


In [11]:
H, status = cv2.findHomography(cam_pts, top_pts)
print("Homography matrix:\n", H)


Homography matrix:
 [[ 7.15992569e-01 -2.81816435e-01 -5.15883420e+00]
 [ 4.85660927e-03  1.67202171e+00 -4.97545242e+02]
 [ 3.69262450e-05 -2.14289277e-04  1.00000000e+00]]


In [13]:
top_img = cv2.imread("Ground Image.png")


In [15]:
tracks = pd.read_csv("tracks.csv")


In [17]:
def bbox_to_point(row):
    x_center = (row.x1 + row.x2) / 2
    y_bottom = row.y2
    return np.array([[x_center, y_bottom]], dtype=np.float32)


In [19]:
def project_point(pt, H):
    pt = np.array([[pt]], dtype=np.float32)  
    mapped = cv2.perspectiveTransform(pt, H)
    return mapped[0][0]


In [25]:
frame_id = 20  # the number which frame you want to plot like bird view
frame_tracks = tracks[tracks.frame_id == frame_id]

vis = top_img.copy()

for _, row in frame_tracks.iterrows():
    pt = bbox_to_point(row)
    x, y = project_point(pt[0], H)

    cv2.circle(vis, (int(x), int(y)), 5, (0,0,255), -1)

cv2.imshow("Bird Eye View", vis)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [29]:
cv2.imwrite("bird_view_pitch.png", vis)


True